<a href="https://colab.research.google.com/github/aritanaoya/football_analysis/blob/main/J1Predict_by_AutoML2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycaret==2.3.1

In [ ]:
from pycaret.utils import enable_colab

enable_colab()

Colab mode enabled.


In [ ]:
import numpy as np
import pandas as pd
import time

csv = '/content/scrapeData20211102.csv'
data = pd.read_csv(csv)
data.shape

(656, 24)

# データの加工

In [ ]:
dataList = ['ＰＫ','セットプレー直接','セットプレーから','クロスから','スルーパスから','ショートパスから','ロングパスから','ドリブルから','こぼれ球から','その他',
            'PK_A','セットプレー直接_A','セットプレーから_A','クロスから_A','スルーパスから_A','ショートパスから_A','ロングパスから_A','ドリブルから_A','こぼれ球から_A','その他_A']

dataList_all= [
               'Unnamed: 0','チーム名','スコア'] + dataList
for item in dataList:   
    data[item] = data[item].str.replace('(', '')
    data[item] = data[item].str.replace(')', '')
    data[item] = data[item].str.replace('%', '').astype(np.float64) / 100
data.set_index('試合日', inplace = True)
data

Unnamed: 0  チーム名  スコア  ...  ドリブルから_A  こぼれ球から_A  その他_A
試合日                                    ...                           
2021-8-9-15:00        home    札幌    2  ...     0.044     0.089  0.022
2021-8-9-15:00        away    浦和    1  ...     0.029     0.118  0.088
2021-8-14-14:00       home    札幌    3  ...     0.044     0.089  0.022
2021-8-14-14:00       away  FC東京    2  ...     0.024     0.143  0.095
2021-8-28-14:00       home    札幌    0  ...     0.044     0.089  0.022
...                    ...   ...  ...  ...       ...       ...    ...
2021-3-21-14:00       away    広島    3  ...     0.000     0.152  0.000
2021-2-27-14:00       home    大分    1  ...     0.041     0.082  0.061
2021-2-27-14:00       away    徳島    1  ...     0.020     0.122  0.061
2021-5-15-16:00       home    大分    1  ...     0.041     0.082  0.061
2021-5-15-16:00       away    鳥栖    1  ...     0.065     0.065  0.097

[656 rows x 23 columns]

In [ ]:
df= pd.concat([data.iloc[::2].reset_index(drop=True), pd.DataFrame(
    data.iloc[1::2].values, columns=dataList_all)], 1)


In [ ]:
filename =  'trainData' + '.csv'
df.to_csv(filename, encoding = 'utf-8-sig') 
data = pd.read_csv('/content/trainData.csv')
data.dtypes

Unnamed: 0        int64
Unnamed: 0.1     object
チーム名             object
スコア               int64
ＰＫ              float64
セットプレー直接        float64
セットプレーから        float64
クロスから           float64
スルーパスから         float64
ショートパスから        float64
ロングパスから         float64
ドリブルから          float64
こぼれ球から          float64
その他             float64
PK_A            float64
セットプレー直接_A      float64
セットプレーから_A      float64
クロスから_A         float64
スルーパスから_A       float64
ショートパスから_A      float64
ロングパスから_A       float64
ドリブルから_A        float64
こぼれ球から_A        float64
その他_A           float64
Unnamed: 0.2     object
チーム名.1           object
スコア.1             int64
ＰＫ.1            float64
セットプレー直接.1      float64
セットプレーから.1      float64
クロスから.1         float64
スルーパスから.1       float64
ショートパスから.1      float64
ロングパスから.1       float64
ドリブルから.1        float64
こぼれ球から.1        float64
その他.1           float64
PK_A.1          float64
セットプレー直接_A.1    float64
セットプレーから_A.1    float64
クロスから_A.1       float64
スルーパスから_A.1     

# SetUP

In [ ]:

data['スコア']

0      1
1      1
2     -2
3      4
4      2
      ..
323    1
324   -3
325   -2
326    0
327    0
Name: スコア, Length: 328, dtype: int64

In [ ]:
from pycaret.regression import setup
data = data.drop('チーム名', axis=1)
def x(a,b):
    return a - b
data['スコア']  = data.apply(lambda f: x(f['スコア'],f['スコア.1']), axis=1)
data = data.drop('スコア.1', axis=1)
#data = data.drop('Unnamed: 0',axis = 1)
setup = setup(data, target='スコア',
        session_id = 123)

Description             Value
0                               session_id               123
1                                   Target               スコア
2                            Original Data         (328, 45)
3                           Missing Values             False
4                         Numeric Features                41
5                     Categorical Features                 3
6                         Ordinal Features             False
7                High Cardinality Features             False
8                  High Cardinality Method              None
9                    Transformed Train Set         (229, 62)
10                    Transformed Test Set          (99, 62)
11                      Shuffle Train-Test              True
12                     Stratify Train-Test             False
13                          Fold Generator             KFold
14                             Fold Number                10
15                                CPU Jobs                -1
16                                 Use GPU             False
17                          Log Experiment             False
18                         Experiment Name  reg-default-name
19                                     USI              9afd
20                         Imputation Type            simple
21          Iterative Imputation Iteration              None
22                         Numeric Imputer              mean
23      Iterative Imputation Numeric Model              None
24                     Categorical Imputer          constant
25  Iterative Imputation Categorical Model              None
26           Unknown Categoricals Handling    least_frequent
27                               Normalize             False
28                        Normalize Method              None
29                          Transformation             False
30                   Transformation Method              None
31                                     PCA             False
32                              PCA Method              None
33                          PCA Components              None
34                     Ignore Low Variance             False
35                     Combine Rare Levels             False
36                    Rare Level Threshold              None
37                         Numeric Binning             False
38                         Remove Outliers             False
39                      Outliers Threshold              None
40                Remove Multicollinearity             False
41             Multicollinearity Threshold              None
42                              Clustering             False
43                    Clustering Iteration              None
44                     Polynomial Features             False
45                       Polynomial Degree              None
46                    Trignometry Features             False
47                    Polynomial Threshold              None
48                          Group Features             False
49                       Feature Selection             False
50                Feature Selection Method           classic
51            Features Selection Threshold              None
52                     Feature Interaction             False
53                           Feature Ratio             False
54                   Interaction Threshold              None
55                        Transform Target             False
56                 Transform Target Method           box-cox

In [ ]:
setup

({'USI',
  'X',
  'X_test',
  'X_train',
  '_all_metrics',
  '_all_models',
  '_all_models_internal',
  '_available_plots',
  '_gpu_n_jobs_param',
  '_internal_pipeline',
  '_ml_usecase',
  'create_model_container',
  'data_before_preprocess',
  'display_container',
  'exp_name_log',
  'experiment__',
  'fix_imbalance_method_param',
  'fix_imbalance_param',
  'fold_generator',
  'fold_groups_param',
  'fold_groups_param_full',
  'fold_param',
  'fold_shuffle_param',
  'gpu_param',
  'html_param',
  'imputation_classifier',
  'imputation_regressor',
  'iterative_imputation_iters_param',
  'log_plots_param',
  'logging_param',
  'master_model_container',
  'n_jobs_param',
  'prep_pipe',
  'pycaret_globals',
  'seed',
  'stratify_param',
  'target_param',
  'transform_target_method_param',
  'transform_target_param',
  'y',
  'y_test',
  'y_train'},
 False,
 [],
 123,
 False,
 False,
 KFold(n_splits=10, random_state=None, shuffle=False),
 'スコア',
 'reg-default-name',
 'lightgbm',
 None,
 0

In [ ]:
from pycaret.regression import compare_models

best_model = compare_models()  # 全てのモデルを訓練し、評価する

Model           MAE           MSE  \
br                         Bayesian Ridge  2.028200e+00  6.559500e+00   
ridge                    Ridge Regression  2.033000e+00  6.593300e+00   
rf                Random Forest Regressor  2.075700e+00  6.736300e+00   
lightgbm  Light Gradient Boosting Machine  0.000000e+00  0.000000e+00   
omp           Orthogonal Matching Pursuit  2.096300e+00  7.085200e+00   
ada                    AdaBoost Regressor  2.132000e+00  7.103000e+00   
huber                     Huber Regressor  2.100500e+00  7.024700e+00   
lasso                    Lasso Regression  2.135500e+00  7.285100e+00   
en                            Elastic Net  2.135500e+00  7.285100e+00   
llar         Lasso Least Angle Regression  2.135500e+00  7.285100e+00   
lr                      Linear Regression  2.114100e+00  7.102900e+00   
et                  Extra Trees Regressor  2.160600e+00  7.343900e+00   
gbr           Gradient Boosting Regressor  2.185300e+00  7.672700e+00   
dt                Decision Tree Regressor  2.843700e+00  1.372270e+01   
par          Passive Aggressive Regressor  3.176100e+00  1.475290e+01   
lar                Least Angle Regression  8.255340e+19  1.979561e+41   

                  RMSE            R2  RMSLE          MAPE  TT (Sec)  
br        2.542400e+00  6.240000e-02    0.0  9.701000e-01     0.017  
ridge     2.548900e+00  5.360000e-02    0.0  9.963000e-01     0.014  
rf        2.583400e+00  1.990000e-02    0.0  1.056200e+00     0.552  
lightgbm  0.000000e+00  0.000000e+00    0.0  0.000000e+00     0.013  
omp       2.645300e+00 -2.130000e-02    0.0  1.053500e+00     0.014  
ada       2.651000e+00 -4.210000e-02    0.0  1.036300e+00     0.113  
huber     2.647200e+00 -4.340000e-02    0.0  1.048600e+00     0.045  
lasso     2.687400e+00 -4.750000e-02    0.0  9.767000e-01     0.016  
en        2.687400e+00 -4.750000e-02    0.0  9.767000e-01     0.016  
llar      2.687400e+00 -4.750000e-02    0.0  9.767000e-01     0.014  
lr        2.657800e+00 -5.300000e-02    0.0  1.069500e+00     0.015  
et        2.699700e+00 -6.910000e-02    0.0  1.112400e+00     0.485  
gbr       2.764500e+00 -1.318000e-01    0.0  1.124600e+00     0.107  
dt        3.674100e+00 -1.005600e+00    0.0  1.495500e+00     0.265  
par       3.688800e+00 -1.280500e+00    0.0  1.930800e+00     0.018  
lar       1.496303e+20 -3.378032e+40    0.0  7.211144e+19     0.028

In [ ]:
print(best_model)

BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, alpha_init=None,
              compute_score=False, copy_X=True, fit_intercept=True,
              lambda_1=1e-06, lambda_2=1e-06, lambda_init=None, n_iter=300,
              normalize=False, tol=0.001, verbose=False)


# Create Model

In [ ]:
from pycaret.regression import create_model

br = create_model("br")  # Bayesian Ridgeのモデルを作成

MAE     MSE    RMSE      R2  RMSLE    MAPE
0     2.2217  9.6781  3.1110 -0.2109   -0.0  0.9315
1     2.3755  7.1505  2.6740  0.1917   -0.0  1.1027
2     2.0392  6.7135  2.5910  0.0334   -0.0  0.9915
3     2.0633  7.4985  2.7383  0.1017   -0.0  0.9517
4     1.9838  6.4140  2.5326  0.1522   -0.0  0.9247
5     2.1685  7.9463  2.8189 -0.1156   -0.0  0.9848
6     1.9390  5.4328  2.3308  0.0717   -0.0  1.1039
7     2.0438  6.2055  2.4911  0.0732   -0.0  1.0052
8     1.6594  4.2470  2.0608  0.1554   -0.0  0.8312
9     1.7878  4.3092  2.0759  0.1710   -0.0  0.8737
Mean  2.0282  6.5595  2.5424  0.0624    0.0  0.9701
SD    0.1957  1.5726  0.3090  0.1242    0.0  0.0836

# Tune Model

In [ ]:
from pycaret.regression import tune_model

tuned_br = tune_model(br)  # ハイパーパラメータの調整

MAE     MSE    RMSE      R2  RMSLE    MAPE
0     2.2204  9.6766  3.1107 -0.2107   -0.0  0.9307
1     2.3701  7.1304  2.6703  0.1940   -0.0  1.0984
2     2.0373  6.7226  2.5928  0.0321   -0.0  0.9911
3     2.0620  7.5025  2.7391  0.1013   -0.0  0.9507
4     1.9859  6.4260  2.5350  0.1506   -0.0  0.9237
5     2.1694  7.9386  2.8175 -0.1145   -0.0  0.9845
6     1.9335  5.4223  2.3286  0.0735   -0.0  1.0977
7     2.0440  6.2000  2.4900  0.0740   -0.0  1.0025
8     1.6543  4.2178  2.0537  0.1612   -0.0  0.8279
9     1.7871  4.3014  2.0740  0.1725   -0.0  0.8728
Mean  2.0264  6.5538  2.5412  0.0634    0.0  0.9680
SD    0.1959  1.5774  0.3103  0.1247    0.0  0.0826

In [ ]:
print(tuned_br)

BayesianRidge(alpha_1=0.0001, alpha_2=1e-07, alpha_init=None,
              compute_score=True, copy_X=True, fit_intercept=False,
              lambda_1=0.001, lambda_2=0.3, lambda_init=None, n_iter=300,
              normalize=True, tol=0.001, verbose=False)


# evalation

In [ ]:
from pycaret.regression import evaluate_model

evaluate_model(tuned_br)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

# deploy

In [ ]:
from pycaret.regression import finalize_model

final_br = finalize_model(tuned_br)
print(final_br)

BayesianRidge(alpha_1=0.0001, alpha_2=1e-07, alpha_init=None,
              compute_score=True, copy_X=True, fit_intercept=False,
              lambda_1=0.001, lambda_2=0.3, lambda_init=None, n_iter=300,
              normalize=True, tol=0.001, verbose=False)


#前回
	Model	MAE	MSE	RMSE	R2	RMSLE	MAPE
0	Bayesian Ridge	0.8964	1.2683	1.1262	0.1359	0.5328	0.3625

#今回
問題点＝＞列ごとの予測になっていて相性的なところが見れていない（チーム毎にどこが相手でも同じスコアになってしまう）


Model	MAE	MSE	RMSE	R2	RMSLE	MAPE
0	Bayesian Ridge	0.8013	1.1055	1.0514	0.1474	0.4946	0.3233

In [ ]:
from pycaret.regression import predict_model

predict_model(final_br);

Model     MAE     MSE    RMSE      R2   RMSLE    MAPE
0  Bayesian Ridge  0.8013  1.1055  1.0514  0.1474  0.4946  0.3233

In [ ]:
from pycaret.regression import save_model

save_model(final_br, "br_model")

Transformation Pipeline and Model Succesfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True, features_todrop=[],
                                       id_columns=[], ml_usecase='regression',
                                       numerical_features=[], target='スコア',
                                       time_features=[])),
                 ('imputer',
                  Simple_Imputer(categorical_strategy='not_available',
                                 fill_value_categorical=None,
                                 fill_value_numerical=None,
                                 numeric_strategy='me...
                 ('clean_names', Clean_Colum_Names()),
                 ('feature_select', 'passthrough'), ('fix_multi', 'passthrough'),
                 ('dfs', 'passthrough'), ('pca', 'passthrough'),
                 ['trained_model',
                  BayesianRidge(alpha_1=0.0001, alpha_2=1e-07, alpha_init=No

In [ ]:
from pycaret.regression import load_model

loaded_br = load_model("br_model")

Transformation Pipeline and Model Successfully Loaded


# get Pred Data

In [ ]:
#試合日は以下を例に入力
#2021-8-9-15:00
def scrapePredData(homeTeamID,awayTeamID,year,matchDate):
    data = pd.DataFrame()
    url = 'https://www.football-lab.jp/comparison/team/' + year + '/' + homeTeamID + '/' + year + '/' + awayTeamID
    #得点パターン、失点パターンを取得
    df1 = pd.read_html(url)[18]
    df1 = df1.drop([0,2,4,6,8,10,12,14,16,18]).drop(df1.columns[[0,2,4,6]], axis=1)


    df2 = pd.read_html(url)[19]
    df2 = df2.drop([0,2,4,6,8,10,12,14,16,18]).drop(df2.columns[[0,2,4,6]], axis=1)
    df2 = df2.replace({'ＰＫ':'PK_A','セットプレー直接':'セットプレー直接_A','セットプレーから':'セットプレーから_A','クロスから':'クロスから_A',
                       'スルーパスから':'スルーパスから_A','ショートパスから':'ショートパスから_A','ロングパスから':'ロングパスから_A','ドリブルから':'ドリブルから_A',
                       'こぼれ球から':'こぼれ球から_A','その他':'その他_A'})

    df = pd.concat([df1,df2])
    df = df.reset_index(drop=True)
    df = df.rename(columns={'割合': 'home','Unnamed: 3': 'Data','割合.1': 'away'})
    df = df[['Data', 'home', 'away']]
    df = df.set_index('Data')
    df = df.T

    time.sleep(1)
    #チーム名と試合日
    homeTeam = get_keys_from_value(TeamDic,homeTeamID)
    awayTeam = get_keys_from_value(TeamDic,awayTeamID)
    df.insert(0, 'チーム名',[homeTeam,awayTeam])
    df.insert(0, '試合日', matchDate)
    data = pd.concat([data, df]) 
    dataList = ['ＰＫ','セットプレー直接','セットプレーから','クロスから','スルーパスから','ショートパスから','ロングパスから','ドリブルから','こぼれ球から','その他',
            'PK_A','セットプレー直接_A','セットプレーから_A','クロスから_A','スルーパスから_A','ショートパスから_A','ロングパスから_A','ドリブルから_A','こぼれ球から_A','その他_A']
    for item in dataList:   
        data[item] = data[item].str.replace('(', '')
        data[item] = data[item].str.replace(')', '')
        data[item] = data[item].str.replace('%', '').astype(np.float64) / 100

    return data


### Team Dic

In [ ]:
TeamDic = {'札幌':'276', '仙台':'238', '鹿島':'120', '浦和':'122', '柏':'132', 'FC東京':'270', '川崎Ｆ':'86', '横浜FM':'124', '横浜FC':'296' , '湘南':'130', '清水':'126', '名古屋':'127', 'Ｇ大阪':'128','Ｃ大阪':'133', '神戸':'136','広島':'129', '徳島':'30116', '福岡':'135', '鳥栖':'269', '大分':'207'}

def get_keys_from_value(d, val):
    return [k for k, v in d.items() if v == val]

### ここのTeamDicと日付を変えれば予想できる

In [ ]:
pred_data = scrapePredData(TeamDic['川崎Ｆ'],TeamDic['仙台'],'2021','2021-11-03-13:00')
pred_data

Data               試合日   チーム名     ＰＫ  ...  ドリブルから_A  こぼれ球から_A  その他_A
home  2021-11-03-13:00  [川崎Ｆ]  0.043  ...     0.095     0.190  0.000
away  2021-11-03-13:00   [仙台]  0.000  ...     0.077     0.115  0.038

[2 rows x 22 columns]

In [ ]:
filename =  'predData' + '.csv'
pred_data.to_csv(filename, encoding = 'utf-8-sig') 
pred_data = pd.read_csv('/content/predData.csv')
pred_data.dtypes

Unnamed: 0     object
試合日            object
チーム名           object
ＰＫ            float64
セットプレー直接      float64
セットプレーから      float64
クロスから         float64
スルーパスから       float64
ショートパスから      float64
ロングパスから       float64
ドリブルから        float64
こぼれ球から        float64
その他           float64
PK_A          float64
セットプレー直接_A    float64
セットプレーから_A    float64
クロスから_A       float64
スルーパスから_A     float64
ショートパスから_A    float64
ロングパスから_A     float64
ドリブルから_A      float64
こぼれ球から_A      float64
その他_A         float64
dtype: object

In [ ]:
# pred_data['チーム名'].astype(str)
# pred_data['チーム名'] = pred_data['チーム名'].str.replace('[', '')
# pred_data['チーム名'] = pred_data['チーム名'].str.replace('\'', '')
# pred_data['チーム名'] = pred_data['チーム名'].str.replace(']', '')
pred_data.set_index('試合日', inplace = True)
pred_data

Unnamed: 0     チーム名     ＰＫ  ...  ドリブルから_A  こぼれ球から_A  その他_A
試合日                                          ...                           
2021-11-03-13:00       home  ['川崎Ｆ']  0.043  ...     0.095     0.190  0.000
2021-11-03-13:00       away   ['仙台']  0.000  ...     0.077     0.115  0.038

[2 rows x 22 columns]

# prediction

In [ ]:

pred_data = pred_data.drop('チーム名', axis=1)
#pred_data = pred_data.drop('Unnamed: 0',axis = 1)
prediction = predict_model(loaded_br, data=pred_data)

In [ ]:
print('Home',prediction['Label'][0],'vs','Away',prediction['Label'][1])

Home 2.0305910173626582 vs Away 0.7024989856156839


川崎　浦和
Home 2.0305910173626582 vs Away 1.217962334660839

浦和　川崎で予想
Home 1.4183249470729138 vs Away 1.8302284049505833

